# Proving Safety of Control Envelopes in KeYmaera X
As a first step, we want to prove the safety of an abstract model of Adaptive Cruise Control

To this end, we prove the safety of a **control envelope** in KeYmaera X.
Contrary to a concrete controller implementation, a control envelope allows for nondeterministic behavior and aims to show that *any* choice permissible according to the nondeterministic program guarantees safety of the system.

Our system will have the following structure:
- The system is described by two variables: Relative (!) Position $p_{\text{rel}}$ and Velocity $v_{\text{rel}}$
- The Neural Network determines the acceleration of the car in the back via discrete control:
  Accelerate ($a_{\text{rel}}=-A$); No Acceleration ($a_{\text{rel}}=0$); Brake ($a_{\text{rel}}=B$)
- The dynamics of the system are given as $p_{\text{rel}}'=v_{\text{rel}},v_{\text{rel}}'=a_{\text{rel}}$ and there is a control decision at least every $0.1$ seconds

We can describe the dynamics of the system as the following hybrid program:
```
{prel'=vrel,vrel'=arel,t'=1&t<=T}
```

Our neural network will have three outputs (`x1`,`x2`,`x3`) and `arel` is then determined based of the maximum of these three cases.
As a control envelope, this can be described as follows:
```
x1:=*;x2:=*;x3:=*;
{
    {?(x1>=x2&x1>=x3);arel:=-A;} ++
    {?(x2>=x1&x2>=x3);arel:=0;} ++
    {?(x3>=x2&x3>=x1);arel:=B;}
}
```
Here `x1:=*` sets x1 to an arbitrary value.
Subsequently we have a nondeterministic choice between 3 programs. Each program checks whether one of the three variables is maximal, e.g. `x1`, and then sets `arel` accordingly.
Note, that this controller is **not** safe yet!
Right now, it admits choosing any of the three accelerations in any of the states.
To turn this into a control envelope with safety guarantees, we have to check whether the chosen acceleration is actually safe.
To this end, we could add a further check after the nondeterministic choice:
```
?(
    prel + T*vrel + arel*T^2/2 >= 0 &
    ((-2*vrel/arel) > T | (prel+(-2*vrel/arel)*vrel + (-2*vrel/arel)^2*arel/2 >= 0))
)
```
The first condition ensures that even following the trajectory with the chosen acceleration `arel` for time `T` still guarantees safety (i.e. the new `prel` is positive).
The second condition considers the case where the sign of our velocity (i.e. `vrel`) changes during the current iteration.
Given the current acceleration, the sign change happens at `(-2*vrel/arel)`.
If this change is within the next iteration (i.e. at time `<=T`) we must ensure that this point also has a positive `prel` (ensured by the second condition).

In principle, we could now just add this second condition to our control envelope and proceed with verification, however, note that `arel` may be `0` which would make the second check unsound.
Instead, in this particular case, we can pull the check into the control envelope program we already have and customize it for the chosen acceleration by replacing `arel` with concrete values:
```
x1:=*;x2:=*;x3:=*;
{
    {
        ?(x1>=x2&x1>=x3);
        ?(
            prel + T*vrel + (-A)*T^2/2 >= (vrel + T*(-A))^2 / (2*B) &
            ((-2*vrel/(-A)) > T | (prel+(-2*vrel/(-A))*vrel + (-2*vrel/(-A))^2*(-A)/2 >= (vrel + (-2*vrel/(-A))*(-A))^2 / (2*B)))
        );
        arel:=-A;
    } ++
    {
        ?(x2>=x1&x2>=x3);
        ?(
            prel + T*vrel >= vrel^2 / (2*B)
        );
        arel:=0;
    } ++
    {
        ?(x3>=x2&x3>=x1);
        ?(
            prel + T*vrel + B*T^2/2 >= (vrel+T*B)^2 / (2*B) &
            ((-2*vrel/B) > T | (prel+(-2*vrel/B)*vrel + (-2*vrel/B)^2*B/2 >= (vrel+(-2*vrel/B)*B)^2 / (2*B)))
        );
        arel:=B;
    }
};
```

With these preparations out of the way, we can now start formalizing our safety guarantee in KeYmaera X.
To this end, we must create a new model of the following structure:
```
ArchiveEntry "ACC-Simple"
ProgramVariables
/* Here will be our program variable definitions */
End.
Definitions
/* Here will be our auxilliary definitions as well as definitions of constants */
End.
Problem
/* Here goes the guarantee we want to prove */
End.
End.
```

### Variables
```
ProgramVariables
/* Network outputs */
Real x1;
Real x2;
Real x3;
/* Relative position of car */
Real prel;
/* Relative velocity of car */
Real vrel;
/* Relative acceleration of car */
Real arel;
/* Clock */
Real t;
End.
```

### Definitions
```
Definitions
/* Time per loop iteration */
Real T;
/* Maximal acceleration */
Real A;
/* Braking acceleration */
Real B;
HP ctrl ::= { /* Here goes the control envelope we devised above. */ }
HP plant ::= { /* Here goes the environment program we devised above */ }
End.
```

### Problem
To formalize the problem we also need to define the considered initial states as well as an invariant that encompasses the safe states of our system.
To this end, consider that our system will always be safe so long as braking with full force avoids a collision. How can this be formaly expressed?
```
/* some preconditons */
->
[{
    ctrl;
    t:=0;
    plant;
}*@invariant(/* some invariant about the system */)] (/* the safety guarantee we want to provide */)
```

### Proof
Once we have all these ingredients together, we can start proving the guarantee in KeYmaera X.
To this end, we must prove that the formula above is a tautology.

For convenience, the full Differential Dynamic Logic Model can be found [here](https://github.com/samysweb/VerSAILLE/tree/main/keymaerax/acc.kyx)